# Libraries

In [12]:
from yahooquery import Ticker
import requests
import json
import yfinance as yf
import pandas as pd
import numpy as np

# Guide to do it with webscraping.
# https://www.scrapehero.com/how-to-scrape-yahoo-finance-data-using-python/
# https://towardsdatascience.com/web-scraping-yahoo-finance-477fe3daa852

# API

In [13]:
# Find ticker symbol by company name.
def ticker(company_name):
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code

partial_name = input("Enter partial company name: ")
stock_info = ticker(partial_name)
print(stock_info)

KO


In [14]:
GetFacebookInformation = yf.Ticker("KO")

# display Company Sector
print("Company Sector : ", GetFacebookInformation.info['sector'])

# display Price Earnings Ratio
print("Price Earnings Ratio : ", GetFacebookInformation.info['trailingPE'])

# display Company Beta
print(" Company Beta : ", GetFacebookInformation.info['beta'])


Company Sector :  Consumer Defensive
Price Earnings Ratio :  26.546255
 Company Beta :  0.546798


# Get the Data

In [22]:
# Get yearly income statemente data for our ticker.
yearly_income_statement = Ticker(stock_info).income_statement(frequency='a')

# Get balance sheet data for our ticker.
yearly_balance_sheet = Ticker(stock_info).balance_sheet(frequency='a')

# Get Cash Flow data for our ticker.
yearly_cash_flow = Ticker(stock_info).cash_flow(frequency='a')

## Clean the dataframes and merge them

In [23]:
# Check which columns have in common income statement and balance sheet.
print(set(yearly_income_statement.columns).intersection(yearly_balance_sheet.columns))

{'currencyCode', 'asOfDate', 'periodType'}


In [24]:
# Check which columns have in common income statement and balance sheet.
print(set(yearly_income_statement.columns).intersection(yearly_cash_flow.columns))

{'currencyCode', 'NetIncome', 'asOfDate', 'periodType'}


In [25]:
# Check which columns have in common income statement and balance sheet.
print(set(yearly_balance_sheet.columns).intersection(yearly_cash_flow.columns))

{'currencyCode', 'asOfDate', 'periodType'}


In [26]:
# Drop periodType, NetIncome and currency code from cash flow.
yearly_cash_flow.drop(['periodType', 'NetIncome', 'currencyCode'], axis=1, inplace=True)

In [27]:
# Drop periodType and Currency code from balance sheet.
yearly_balance_sheet.drop(['periodType', 'currencyCode'], axis=1, inplace=True)

In [28]:
# Join the 3 dataframes into one.
# Using 'asOfDate' as key to join the dataframes.
# When you have two equals columns, use suffixes to differentiate them.
merged_dataframe = pd.merge(yearly_income_statement, yearly_balance_sheet, on='asOfDate')

# Create the dataframe that we will work on.

In [29]:
# # Create a dataframe with 6 columns: TTM, 2019-12-31, 2020-12-31, 2021-12-31, 2022-12-31, 2023-12-31
# # and 1 row for each column of the income statement.
# df = pd.DataFrame(columns=['2019-12-31', '2020-12-31', '2021-12-31', '2022-12-31'],
#                     index=merged_dataframe.columns)
# df

,2019-12-31,2020-12-31,2021-12-31,2022-12-31
asOfDate,NaN,NaN,NaN,NaN
periodType,NaN,NaN,NaN,NaN
currencyCode,NaN,NaN,NaN,NaN
BasicAverageShares,NaN,NaN,NaN,NaN
BasicEPS,NaN,NaN,NaN,NaN
...,...,...,...,...
TotalNonCurrentLiabilitiesNetMinorityInterest,NaN,NaN,NaN,NaN
TotalTaxPayable,NaN,NaN,NaN,NaN
TreasurySharesNumber,NaN,NaN,NaN,NaN
TreasuryStock,NaN,NaN,NaN,NaN


In [87]:
df = merged_dataframe.copy()

In [88]:
df = df[['asOfDate', 'BasicEPS', 'EBIT', 'GrossProfit', 'NetIncome', 'OperatingIncome', 'TotalRevenue', 'SalariesAndWages', 'TotalExpenses', 'TotalAssets', 'TotalDebt', 'StockholdersEquity']]

In [89]:
# Drop last row of the dataframe.
df.drop(df.tail(1).index, inplace=True)

In [90]:
df.set_index('asOfDate', inplace=True)

In [91]:
df = df.transpose()

In [92]:
df

asOfDate,2019-12-31,2020-12-31,2021-12-31,2022-12-31
BasicEPS,2.090000e+00,1.800000e+00,2.260000e+00,2.200000e+00
EBIT,1.173200e+10,1.118600e+10,1.402200e+10,1.256800e+10
GrossProfit,2.264700e+10,1.958100e+10,2.329800e+10,2.500400e+10
NetIncome,8.920000e+09,7.747000e+09,9.771000e+09,9.542000e+09
OperatingIncome,1.008600e+10,8.997000e+09,1.030800e+10,1.090900e+10
TotalRevenue,3.726600e+10,3.301400e+10,3.865500e+10,4.300400e+10
SalariesAndWages,2.010000e+08,1.260000e+08,3.370000e+08,3.560000e+08
TotalExpenses,2.718000e+10,2.401700e+10,2.834700e+10,3.209500e+10
TotalAssets,8.638100e+10,8.729600e+10,9.435400e+10,9.276300e+10
TotalDebt,4.276300e+10,4.279300e+10,4.276100e+10,3.914900e+10


In [33]:
# Let's fill the dataframe with the data from merged_dataframe.
# We will iterate over each value using the column for df and the index, and the value will come from merged_dataframe using 

In [34]:
P/E
P/B
Debt-to-equity
Free Cash Flow
Dividend Yield
PEG ration
ROE
ROA
ROI
EPS
Beta
Market Cap
Enterprise Value
EV/EBITDA
EV/EBIT
EV/FCF
EV/Sales
EBITDA
EBIT
Liquidity
Current Ratio
Quick Ratio
Interest Coverage
Debt-to-equity
Debt-to-asset
Debt-to-capital
Debt-to-EBITDA
Debt-to-EBIT
Debt-to-FCF
Debt-to-sales
Debt-to-equity
Debt-to-asset
Debt-to-capital

Current Debt
Long-term Debt
Short-term Debt
Total Debt

Equity
Total Assets
Current Assets
Cash

Y/Y % Change
5Y Avg. % Change
10Y Avg. % Change

5 Years CAGR
10 Years CAGR
Lifetime CAGR

Earnings Date

PEG Ratio (5 yr expected)	




SyntaxError: invalid decimal literal (3524191168.py, line 46)

: 